## 16. Defining clear function contracts: pre‑conditions / post‑conditions

Every callable should advertise **what it expects** (pre‑conditions) and **what it guarantees** (post‑conditions).  You can document these in docstrings, enforce them with `assert`, or encode them as type hints.  

Benefits:
• Makes the function predictable for callers.  
• Lets tests focus on behaviour, not implementation.  
• Encourages small, single‑purpose functions.

Common gotcha → hidden side‑effects that violate the contract (*modifies global state*).

```python
def divide(a: float, b: float) -> float:
    """Return a / b.
    Pre: b ≠ 0
    Post: result * b == a (within float eps)"""
    if b == 0:
        raise ValueError('b must be non‑zero')
    return a / b
```

### Quick check

1. True / False Documenting side‑effects is part of a function contract.

2. A post‑condition is checked **after** execution to ensure:
  a. inputs valid  b. output meets guarantee

<details><summary>Answer key</summary>

1. **True** — side‑effects influence callers.
2. **b**

</details>

## 17. Choosing between composition and inheritance

Inheritance expresses an **is‑a** relationship; composition an **has‑a**.  Prefer composition when you only need to reuse behaviour, not the type identity.  Deep inheritance trees become brittle; swapped components via composition keep systems flexible.

```python
class Engine:
    def start(self): print('vroom')

# Composition
class Car:
    def __init__(self, engine: Engine):
        self.engine = engine
    def drive(self):
        self.engine.start(); print('go')

# Inheritance misuse
class CarEngine(Engine):
    pass  # Car is not a kind of Engine!
```

### Quick check

1. Which is safer for swapping implementations at runtime?
  a. inheritance  b. composition

2. True / False Inheritance automatically exposes all parent public methods.

<details><summary>Answer key</summary>

1. **b** — inject new component.
2. **True** — unless deliberately hidden, all are inherited.

</details>

## 18. Data‑oriented design vs behaviour‑oriented design

*Data‑oriented*: emphasise plain data structures (dicts, tuples) + separate processing functions.  Great for analytics, functional pipelines, concurrency.

*Behaviour‑oriented* (OO): bundle data *and* behaviour inside objects.  Useful when invariants must be enforced or when polymorphism matters.

Choosing:
• High throughput over huge homogeneous data? → data‑oriented.  
• Rich domain rules, needing method polymorphism? → behaviour‑oriented.

```python
# Data‑oriented ETL row
row = {'id': 1, 'kwh': 42}
def bill(row): return row['kwh'] * 0.15

# Behaviour‑oriented domain model
class MeterReading:
    rate = 0.15
    def __init__(self, kwh): self.kwh = kwh
    def bill(self): return self.kwh * self.rate
```

### Quick check

1. Data‑oriented design favours:
  a. cache‑friendly loops  b. polymorphic dispatch

2. True / False Behaviour‑oriented design makes it easier to enforce invariants.

<details><summary>Answer key</summary>

1. **a**
2. **True**

</details>

## 19. Error‑handling strategy: fail fast vs fail safe

**Fail fast**: stop at first sign of invalid state, raise error loudly.  Valuable in core business rules where silent corruption is unacceptable.  

**Fail safe**: continue with degraded service — e.g., retry, default values, circuit breakers.  Useful at system boundaries (network glitches, user typos).  

Decide per layer: domain logic → fail fast, integration/adapters → fail safe.

```python
# fail fast
price = db.fetch_price('SKU1')
if price is None:
    raise LookupError('Missing price!')

# fail safe wrapper
try:
    send_email(msg)
except TimeoutError:
    log.warning('Email service down, queuing retry')
    queue_retry(msg)
```

### Quick check

1. Which layer typically adopts **fail safe**?
  a. core domain math  b. HTTP client adapter

2. True / False Fail fast means masking the error and moving on.

<details><summary>Answer key</summary>

1. **b**
2. **False** — fail fast surfaces error immediately.

</details>

## 20. Using assertions vs raising exceptions

`assert` is for **internal sanity checks** — conditions that should never happen if your code is correct.  They can be stripped with the `-O` flag, so never rely on them for user‑facing validation.  

Use `raise` to signal **recoverable errors** caused by external factors (bad input, missing file).

```python
def sqrt(x):
    assert x >= 0, 'x must be non‑negative'  # dev check
    return x ** 0.5

def read_file(path):
    try:
        with open(path) as f:
            return f.read()
    except FileNotFoundError as e:
        raise e  # proper exception
```

### Quick check

1. Assertions may be skipped when Python runs with:
  a. -O   b. default flags

2. True / False Use `assert` to validate user form inputs.

<details><summary>Answer key</summary>

1. **a**
2. **False** — raise exceptions: assertions might be stripped.

</details>